In [1]:
from utils_data_prep import *
import glob
import os
import geopandas as gpd

import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import numpy as np

# KyFromAbove Tile Index

* The KyFromAbove 5K tile index contains three polygon feature classes for downloading the DEM, Aerial Imagery, and LiDAR point cloud datasets. Each feature class has attributes including the available datasets for each tile (some tiles have had lidar and/or imagery collected over multiple years) and data download links.

    * Note: The KyFromAbove aerial imagery is three bands (RGB), and the fourth infrared band dataset must be downloaded from the REST API without the tile index.

* The code below...
    1. The tile index is downloaded and each of the three feature class layers are read and saved as GeoJSON files. 
    2. GeoJSON tile index files are then used to extract only the relevant tiles that cover the Warren or Hardin County areas.

In [ ]:
################################
# Kentucky index tile dataset
################################

# download path for tile index geodatabase zip file
download_path = r'../data/KyFromAbove_Data_Product_Tile_Grids.gdb.zip'


###### get tile index files or handle existing
if len(glob.glob(r'../data/*.geojson')) > 0:
    for file in glob.glob(r'../data/*.geojson'):
        print(f"File - {file} - already exists...")
else:
    # custom function to download tile index, read geodatabase, and save feature classes as geojsons
    get_tile_index(download_path)

# delete point cloud index file
for file in glob.glob(r'../data/*.geojson'):
    if 'pointcloud' in file.lower():
        os.remove(file)

## *Warren County*

In [ ]:
###########################################################
# Warren County dataset boundary & buffered boundary
###########################################################

# path to geodatabase containing boundary feature class
gdb_path = r'../data/geology.gdb'

# layer name of boundary feature class
boundary_layer = r'warren_geo_boundary'

# path to save output as GeoJSON
boundary_output_path = r'../data/warren/boundary.geojson'

# set buffer distance in spatial units of feature class (feet in this case)
# buffer distance should be enough to mitigate edge effects of any DEM derivative terrain feature calculations
buffer_distance = 1000

# path to save buffered output as GeoJSON
buffered_output_path = f"../data/warren/boundary_buffer{buffer_distance}.geojson"


##### create boundary GeoJSON if it doesn't exist
if os.path.isfile(boundary_output_path):
    print(f"File - {boundary_output_path} - already exists")
else:
    gdf = gpd.read_file(gdb_path, layer=boundary_layer)
    gdf.to_file(boundary_output_path, driver='GeoJSON')


##### create buffered boundary GeoJSON if it doesn't exist
if os.path.isfile(buffered_output_path):
    print(f"File - {buffered_output_path} - already exists")
else:
    gdf = gpd.read_file(gdb_path, layer=boundary_layer)
    gdf['geometry'] = gdf['geometry'].buffer(buffer_distance)
    gdf.to_file(buffered_output_path, driver='GeoJSON')

In [ ]:
##############################################
# extract tiles that intersect warren county
##############################################

# list of tile index geojson file paths
index_paths = glob.glob(r'../data/*.geojson')

# path to labeled dataset boundary file (use buffered boundary)
boundary_path = glob.glob(r'../data/warren/boundary_buffer*.geojson')[0]


###### iterate through each index (should be dem & aerial)
for path in index_paths:

    # get filename of index
    filename = os.path.basename(path) 
    
    # create output path
    output_path = f"../data/warren/warren_{filename}"

    if os.path.isfile(output_path):
        print(f"File - {output_path} - already exists...")
    else:
        # custom function to get tiles intersecting boundary and save as output geojson
        get_intersecting_index_tiles(path, boundary_path, output_path)

In [ ]:
##################################################
# visualize coverage of index tiles and datasets
##################################################

# boundary path
boundary_path = r'../data/warren/boundary.geojson'

# buffered boundary path
buffered_path = glob.glob(r'../data/warren/boundary_buffer*.geojson')[0]

# get list of index geojsons for warren county (excluding buffered boundary geojson)
index_paths = [file for file in glob.glob(r'../data/warren/*.geojson') if not 'boundary' in file]

print(index_paths)
###### plot boundaries and index tiles...
fig, ax = plt.subplots(ncols=2, figsize=(10,8))

# read boundares as geodataframes
boundary = gpd.read_file(boundary_path)
buffered = gpd.read_file(buffered_path)

# iterate through list of geojson index paths (dem & aerial)
for idx, path in enumerate(index_paths):
    index = gpd.read_file(path)
    boundary.plot(ax=ax[idx])
    buffered.plot(ax=ax[idx], edgecolor='red', facecolor='none')
    index.plot(ax=ax[idx], edgecolor='k', facecolor='none')
    ax[idx].set_title(os.path.basename(path), style='italic', fontsize=8)
    ax[idx].set_axis_off()

plt.tight_layout()
plt.show()

## *Hardin County*

# DEM

* The Kentucky statewide digital elevation model (DEM) is derived from airborne lidar collected since 2010. The data collection prograrm has been divided into phases based on time and resolution, with Phase 1 resulting in a 5 foot resolution DEM, and a 2 foot resolution DEM for Phase 2. The data are publicly available as downloads of small tiles in either .img (with associated metadata files) or .tif (GeoTIFF).

* The code below...
    1. The tile index grids are used to download the relevant individual .tif files for the 5 foot DEM (Phase 1) in the Warren and Hardin County datasets.
    2. Individual DEM tiles are then mosaiced into a single DEM for each area and saved as a new GeoTIFF.
    3. The single DEMs for each area are finally clipped to the project area extents of Warren and Hardin Counties. 

## *Warren County*

In [ ]:
###############################################
# download DEM tiles (.tif) for Warren County
###############################################

# path to geojson dem tile index
index_path = r'../data/warren/warren_KYAPED_5FT_DEM_Tile_Index.geojson'

# field containing unique tile id for file naming
id_field = 'TileName'

# field containing geotiff download url
url_field = 'Phase1_AWS_url'

# output directory to save downloaded tiles
output_dir = r'../data/warren/dem_tiles'


###### call custom function to download tiles
download_data_tiles(index_path, id_field, url_field, output_dir)

In [ ]:
#############################################
# Mosaic DEM tiles into single DEM GeoTIFF
#############################################

# path to buffered boundary geojson file
boundary_path = glob.glob(r'../data/warren/boundary_buffer*.geojson')[0]

# path to geojson containing tile polygons
index_path = r'../data/warren/warren_KYAPED_5FT_DEM_Tile_Index.geojson'

# directory containing dem tiles
dem_tile_dir = r'../data/warren/dem_tiles'

# path for output dem
output_dem_path = r'../data/warren/dem.tif'


##### get lists of paths of edge tiles & contained tiles
within_tile_paths, edge_tile_paths = get_contained_and_edge_tile_paths(index_path, boundary_path, dem_tile_dir)


##### iterate through edge tiles and clip to dataset boundary
for tile_path in edge_tile_paths:
    clip_image_to_boundary(tile_path, boundary_path, output_tif_path=None)


##### get list of clipped edge tile paths & combine with contained tile paths
clipped_edge_tile_paths = glob.glob(f"{dem_tile_dir}/*clip.tif")
tile_paths_list = within_tile_paths + clipped_edge_tile_paths


##### mosaic clipped edge and contained tiles into single dem and save
mosaic_image_tiles(tile_paths_list, output_dem_path)


##### clean up clipped tiles (keep original full tiles)
for path in clipped_edge_tile_paths:
    os.remove(path)

In [ ]:
##########################
# visualize clipped dem
##########################

dem_path = r'../data/warren/dem.tif'

fig, ax = plt.subplots(figsize=(10,10))

with rasterio.open(dem_path) as src:
    data = src.read(1, masked=True)
    min_val = np.min(data)
    max_val = np.max(data)
    dem = ax.imshow(data, cmap='viridis', interpolation='bicubic')
    src.close()

cbar = fig.colorbar(dem, ax=ax, orientation='vertical', shrink=0.5)
cbar.set_ticks([min_val, max_val])
cbar.set_label('Elevation (feet)', labelpad=-70)

ax.set_title('Warren County DEM')
ax.set_xlabel('X (pixels)')
ax.set_ylabel('Y (pixels)')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_position(('outward', 10))  # offset bottom axis
ax.spines['left'].set_position(('outward', 10))    # offset left axis
    
plt.tight_layout()
plt.show()

# ax.set_axis_off()
# plt.savefig(r'../data/warren/dem.png', bbox_inches='tight', pad_inches=0, transparent=True)

## *Hardin County*

# Aerial Imagery

* Aerial imagery has been collected simultaneously with lidar through the KyFromAbove program with resolutions between 1 foot to 3 inches, and complete statewide coverage of 6 inches. Imagery was collected for true color imagery with three bands (RGB), but near infrared has been captured as a fourth band in some areas; however, only true color imagery is available for download.

* The code below...
    1. The tile index grids are used to download individual aerial imagery tiles.
    2. Individual tiles are mosaiced together and then saved as a single GeoTIFF image.
    3. The single aerial imagery files are then clipped to the boundaries of the Warren and Hardin Counties datasets.

## *Warren County*

In [2]:
###################################################
# Download aerial imagery tiles for Warren County
###################################################

# path to geojson aerial imagery tile index
index_path = r'../data/warren/warren_KYAPED_Aerial_Tile_Index.geojson'

# field containing unique tile id for file naming
id_field = 'TileName'

# field containing geotiff download url (only available column for image downloads, 6 inch resolution)
url_field = 'Boxzip2021'

# output directory to save downloaded tiles
output_dir = r'../data/warren/aerial_tiles'

# call custom function to download tiles
download_data_tiles(index_path, id_field, url_field, output_dir)

In [ ]:
with rasterio.open(r'../data/warren/dem.tif') as src:
    print(src.meta)
    print(src.dtypes)

In [2]:
def get_contained_and_edge_tile_paths(index_path, boundary_path, data_dir):

    gdf_index = gpd.read_file(index_path)
    gdf_boundary = gpd.read_file(boundary_path)

    if gdf_boundary.crs != gdf_index.crs:
        gdf_index = gdf_index.to_crs(gdf_boundary.crs)
    
    boundary = gdf_boundary.iloc[0].geometry
    within_polygons = gdf_index[gdf_index.geometry.within(boundary)]
    edge_polygons = gdf_index[~gdf_index.index.isin(within_polygons.index)]

    within_poly_paths = []
    for _, row in within_polygons.iterrows():
        tile = row['TileName']
        path = glob.glob(f"{data_dir}/*{tile}*.tif")[0]
        within_poly_paths.append(path)

    edge_poly_paths = []
    for _, row in edge_polygons.iterrows():
        tile = row['TileName']
        path = glob.glob(f"{data_dir}/*{tile}*.tif")[0]
        edge_poly_paths.append(path)

    return within_poly_paths, edge_poly_paths

In [3]:
from rasterio.mask import mask


def clip_image_to_boundary(input_tif_path, boundary_path):
    
    if '_clip.tif' in input_tif_path:
        return

    boundary = gpd.read_file(boundary_path)

    with rasterio.Env(CHECK_DISK_FREE_SPACE='FALSE'):
        with rasterio.open(input_tif_path) as src:
            
            data_type = src.meta['dtype']
            if src.nodata is None:
                nodata_value = np.nan
                data_type = 'float32'
            else:
                nodata_value = src.nodata

            out_image, out_transform = mask(src, shapes=boundary.geometry, crop=True, nodata=nodata_value)
            out_meta = src.meta.copy()
            out_meta.update({'driver':'GTiff', 
                            'height':out_image.shape[1], 
                            'width':out_image.shape[2], 
                            'transform':out_transform, 
                            'crs': src.crs, 
                            'nodata': nodata_value, 
                            'dtype': data_type})
            
            output_dir = os.path.dirname(input_tif_path)
            output_filename = os.path.splitext(os.path.basename(input_tif_path))[0] + '_clip.tif'
            output_path = os.path.join(output_dir, output_filename)

            with rasterio.open(output_path, 'w', **out_meta) as output:
                for i in range(out_image.shape[0]):
                    output.write(out_image[i, :, :], i+1)
        src.close()

In [4]:
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling


def mosaic_resample_reproject(tiles_paths, channel, reference_path, output_path):

    images = [rasterio.open(path, 'r') for path in tiles_paths]

    mosaic, mosaic_transform = merge(images, indexes=[channel])
    print('Mosaic shape: ', mosaic.shape)

    with rasterio.open(reference_path) as ref:

        ref_transform, ref_width, ref_height = calculate_default_transform(images[0].crs, 
                                                                           ref.crs, 
                                                                           mosaic.shape[2], 
                                                                           mosaic.shape[1], 
                                                                           *ref.bounds, 
                                                                           dst_width=ref.width,
                                                                           dst_height=ref.height)
        
        empty_array = np.zeros((ref.count, ref_height, ref_width), dtype=ref.dtypes[0])

    output_meta = ref.meta.copy()
    output_meta.update({'driver':'GTiff', 
                        'transform':ref_transform, 
                        'width':ref_width, 
                        'height': ref_height, 
                        'count':1})
    
    print('Updated metadata...')

    with rasterio.open(output_path, 'w', **output_meta) as dst:

        reproject(source=mosaic, 
                  destination=empty_array, 
                  src_transform=mosaic_transform, 
                  src_crs=images[0].crs, 
                  dst_transform=ref_transform, 
                  dst_crs=ref.crs, 
                #   dst_width=ref_width, 
                #   dst_height=ref_height, 
                  resampling=Resampling.bilinear)
        
        dst.write(empty_array, 1)

In [5]:

aerial_tiles_dir = r'../data/warren/aerial_tiles'

index_path = glob.glob(r'../data/warren/*Aerial*.geojson')[0]

boundary_path = glob.glob(r'../data/warren/boundary_buffer*.geojson')[0]

reference_path = r'../data/warren/dem.tif'


##### convert dtypes to float32
for path in glob.glob(f"{aerial_tiles_dir}/*.tif"):
    if '_f32' in path:
        continue
    else:
        convert_image_dtype(path)
print('Converted dtypes...')

##### delete original UINT8 files & keep float32 files
all_tile_paths = glob.glob(f"{aerial_tiles_dir}/*.*")
for path in all_tile_paths:
    if '_f32' in path:
        continue
    elif '_clip' in path:
        continue
    else:
        os.remove(path)
print('Deleted non-converted files...')

##### get lists of tiles along boundary edge & within boundary area
contained_tile_paths, edge_tile_paths = get_contained_and_edge_tile_paths(index_path, boundary_path, aerial_tiles_dir)
print('Got lists...')


# ##### clip edge tiles and save as new tiles
# for tile_path in edge_tile_paths:
#     clip_image_to_boundary(tile_path, boundary_path)
# print('Clipped edge tiles...')


##### combine contained tile paths with new clipped tile paths
clipped_tile_paths = glob.glob(f"{aerial_tiles_dir}/*_clip.tif")
tile_paths = contained_tile_paths + clipped_tile_paths
print('Got complete list of tiles for mosaic...')


##### mosaic tiles, resample mosaic, reproject mosaic, & save final file
band_names = ['red', 'green', 'blue', 'ir']
for i, band in enumerate(band_names, start=1):
    output_path = f"../data/warren/aerial_{band}_5ft.tif"
    mosaic_resample_reproject(tile_paths, i, reference_path, output_path)
    print('Final step...')

    break


Converted dtypes...
Deleted non-converted files...
Got lists...
Got complete list of tiles for mosaic...
Mosaic shape:  (1, 187222, 224365)
Updated metadata...


: 

In [ ]:
#########################
# Convert RGB+IR dtypes
#########################

# paths for aerial tiles
aerial_tile_paths = glob.glob(r'../data/warren/aerial_tiles/*.tif')

# custom function for converting aerial image tiles to float32 with nan for nodata
for path in aerial_tile_paths:
    convert_image_dtype(path, 'float32')

In [ ]:
############################################################
# Mosaic aerial imagery tiles into single channel GeoTIFFs
############################################################

# path to geojson containing tile polygons
index_path = r'../data/warren/warren_KYAPED_Aerial_Tile_Index.geojson'

# path to buffered boundary geojson file
boundary_path = glob.glob(r'../data/warren/boundary_buffer*.geojson')[0]

# directory containing aerial tiles
aerial_tile_dir = r'../data/warren/aerial_tiles'


# get lists of paths of edge tiles & contained tiles
within_tile_paths, edge_tile_paths = get_contained_and_edge_tile_paths(index_path, boundary_path, aerial_tile_dir, file_suffix='_f32.tif')


# iterate through edge tiles and clip to dataset boundary
for tile_path in edge_tile_paths:
    clip_image_to_boundary(tile_path, boundary_path, output_tif_path=None)


# get list of clipped edge tile paths & combine with contained tile paths
clipped_edge_tile_paths = glob.glob(f"{aerial_tile_dir}/*clip.tif")
tile_paths_list = within_tile_paths + clipped_edge_tile_paths


# iterate through bands and mosaic clipped edge and contained tiles into single image and save...
band_names = ['red', 'green', 'blue', 'ir']
for i, band in enumerate(band_names, start=1):
    output_aerial_path = f"../data/warren/aerial_{band}_5ft.tif"
    mosaic_image_tiles(tile_paths_list, output_aerial_path, band_number=i, resample=(5,5))


# clean up clipped tiles (keep original tiles)
for path in clipped_edge_tile_paths:
    os.remove(path)

In [ ]:
####################################
# Visualize clipped aerial images
####################################

# paths to single band aerial images
red_path = r'../data/warren/aerial_red_5ft.tif'
green_path = r'../data/warren/aerial_green_5ft.tif'
blue_path = r'../data/warren/aerial_blue_5ft.tif'
ir_path = r'../data/warren/aerial_ir_5ft.tif'
aerial_paths = [red_path, green_path, blue_path, ir_path]

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15,13))
ax = ax.ravel()

for idx, path in enumerate(zip(aerial_paths, ['Reds', 'Greens', 'Blues', 'coolwarm'])):
    with rasterio.open(path[0]) as src:
        data = src.read(1)
        ax[idx].imshow(data, cmap=path[1], interpolation='bicubic')
        ax[idx].set_title(['Red', 'Green', 'Blue', 'Infrared'][idx], style='italic')
        ax[idx].set_axis_off()
        src.close()

        # fig, ax = plt.subplots(figsize=(10,10))
        # ax.imshow(data, cmap=path[1], interpolation='bicubic')
        # ax.set_axis_off()
        # img_name = ['Red', 'Green', 'Blue', 'Infrared'][idx]
        # plt.savefig(f"../data/warren/aerial_{img_name}_5ft.png", bbox_inches='tight', pad_inches=0, transparent=True)

plt.suptitle('Warren County Aerial RGB+IR Imagery')
plt.tight_layout()
plt.show()

## Hardin County

# Hydrography

The NHDPlus HR is a geospatial dataset depicting the flow of water across the Nation's landscapes and through the stream network. The NHDPlus HR was built using the National Hydrography Dataset High Resolution data at 1:24,000 scale or more detailed, the 10 meter 3D Elevation Program data, and the nationally complete Watershed Boundary Dataset. For more details, see: https://pubs.usgs.gov/publication/ofr20191096.

## Warren County

In [ ]:
################################################
# Download NHDPlus HR Dataset for Warren County
################################################

# url for HUC 4 Basin 0511 (covers Warren County area)
nhd_url = r'https://prd-tnm.s3.amazonaws.com/StagedProducts/Hydrography/NHDPlusHR/VPU/Current/GDB/NHDPLUS_H_0511_HU4_GDB.zip'

# directory to save hydrography
nhd_dir = r'../data/warren/nhdplushr'
if not os.path.isdir(nhd_dir):
    os.makedirs(nhd_dir)

# path to save NHDPlus HR geodatabase
nhd_output_path = r'../data/warren/nhdplushr/nhd.zip'

# custom function to download zip, extract contents, and delete zip
download_zip(nhd_url, nhd_output_path)

In [ ]:
###################################################################
# Clip to Flowline & Waterbody feature classes to Dataset Boundary
###################################################################

# path to NHDPlus HR geodatabase
nhd_gdb_path = glob.glob(r'../data/warren/nhdplushr/*.gdb')[0]

# path to dataset boundary GeoJSON
boundary_path = glob.glob(r'../data/warren/*boundary*.geojson')[0]

# iterate through specified layers in geodatabase and clip to dataset boundary
# (NHDFlowline & NHDWaterbody in Hydrography feature classes for this project)
for gdb_layer in ['NHDFlowline', 'NHDWaterbody']:
    output_path = f"../data/warren/nhdplushr/{gdb_layer}.geojson"
    if not os.path.isfile(output_path):
        clip_spatial_to_boundary(nhd_gdb_path, boundary_path, output_path, gdb_layer)

# delete geodatabase after clipping to dataset boundary
shutil.rmtree(nhd_gdb_path)

In [ ]:
####################
# Convert to Images
####################

nhd_gis_files = glob.glob(r'../data/warren/nhdplushr/*.geojson')
reference_image = r'../data/warren/dem.tif'

for nhd_path in nhd_gis_files:
    output_filename = os.path.basename(nhd_path)
    output_filename = os.path.splitext(output_filename)[0]
    output_path = f"../data/warren/{output_filename}.tif"
    convert_spatial_to_reference_image(nhd_path, reference_image, output_path, attribute=None)

In [ ]:
#######################################
# Visualize Flowline & Waterbody (GIS) 
#######################################

fig, ax = plt.subplots(figsize=(7, 7))

gdf_flowline = gpd.read_file(r'../data/warren/nhdplushr/NHDFlowline.geojson')
gdf_waterbody = gpd.read_file(r'../data/warren/nhdplushr/NHDWaterbody.geojson')
gdf_boundary = gpd.read_file(glob.glob(r'../data/warren/*boundary*.geojson')[0])

gdf_boundary.plot(ax=ax, color='grey')
gdf_flowline.plot(ax=ax, color='#6EACDA', linewidth=1)
gdf_waterbody.plot(ax=ax, color='#03346E')

ax.set_axis_off()
ax.set_title('Warren County, NHDPlus HR Dataset\n(NHDFlowline & NHDWaterbody)', y=0.95)
plt.tight_layout()
plt.show()

In [ ]:
###########################################
# Visualize Flowline & Waterbody (Images) 
###########################################

flowline_path = r'../data/warren/NHDFlowline.tif'

waterbody_path = r'../data/warren/NHDWaterbody.tif'

fig, ax = plt.subplots(ncols=2, figsize=(12,6))

with rasterio.open(flowline_path, 'r') as flowline:
    show(flowline, ax=ax[0])

with rasterio.open(waterbody_path, 'r') as waterbody:
    show(waterbody, ax=ax[1])

for axes in ax:
    axes.set_axis_off()
ax[0].set_title('Flowline', style='italic')
ax[1].set_title('Waterbody', style='italic')
plt.suptitle('NHDPlus HR Dataset - Warren County', y=0.95)
plt.tight_layout()
plt.show()

## Hardin County

In [ ]:
# url's for NHDPlusHR HUC4 Basins covering Hardin County

# nhd_north_url = r'https://prd-tnm.s3.amazonaws.com/StagedProducts/Hydrography/NHDPlusHR/VPU/Current/GDB/NHDPLUS_H_0514_HU4_GDB.zip'

# nhd_south_url = r'https://prd-tnm.s3.amazonaws.com/StagedProducts/Hydrography/NHDPlusHR/VPU/Current/GDB/NHDPLUS_H_0511_HU4_GDB.zip'

# Roads & Railroads

In [ ]:
#############################################
# Download & save OSM files for Kentucky
#############################################

# URL for Open Street Maps Kentucky dataset download
osm_url = r'http://download.geofabrik.de/north-america/us/kentucky-latest-free.shp.zip'

# directory to save Open Street Maps dataset
osm_dir = r'../data/osm_ky'
if not os.path.isdir(osm_dir):
    os.makedirs(osm_dir)

# path to temporarily save downloaded zip file
osm_zip_path = r'../data/osm_ky/osm.zip'

# custom function to download zip, extract contents, and delete zip
download_zip(osm_url, osm_zip_path)

# only save shapefiles for roads and railroads (gis_osm_roads_free_1 & gis_osm_railways_free_1)
all_shapefiles = glob.glob(os.path.join(osm_dir, '*.*'))
for path in all_shapefiles:
    if 'gis_osm_roads_free_1' in path:
        continue
    elif 'gis_osm_railways_free_1' in path:
        continue
    else:
        os.remove(path)

## Warren County

In [ ]:
########################################################################
# Clip to Roads & Railways shapefiles to Warren County Dataset Boundary
########################################################################

# paths to directory containing road and railway shapefiles
osm_shp_paths = glob.glob(r'../data/osm_ky/*.shp')

# path to dataset boundary GeoJSON
boundary_path = glob.glob(r'../data/warren/boundary_buffer*.geojson')[0]

# directory to store clipped osm datasets
osm_warren_dir = r'../data/warren/osm_warren'
if not os.path.isdir(osm_warren_dir):
    os.makedirs(osm_warren_dir)

# iterate through shapefiles and clip to dataset boundary
for shp_path in osm_shp_paths:
    filename = os.path.basename(shp_path)
    filename = os.path.splitext(filename)[0]
    if 'roads' in filename:
        filename = 'OSM_Roads'
    else:
        filename = 'OSM_Railways'
    output_path = f"{osm_warren_dir}/{filename}.geojson"
    if not os.path.isfile(output_path):
        clip_spatial_to_boundary(shp_path, boundary_path, output_path)

In [ ]:
####################
# Convert to Images
####################

osm_warren_files = glob.glob(r'../data/warren/osm_warren/*.geojson')

reference_image = r'../data/warren/dem.tif'

for osm_path in osm_warren_files:
    output_filename = os.path.basename(osm_path)
    output_filename = os.path.splitext(output_filename)[0]
    output_path = f"../data/warren/{output_filename}.tif"
    convert_spatial_to_reference_image(osm_path, reference_image, output_path, attribute=None)

In [ ]:
#######################################
# Visualize Roads & Railways (GIS) 
#######################################

fig, ax = plt.subplots(figsize=(7, 7))

gdf_roads = gpd.read_file(r'../data/warren/osm_warren/OSM_Roads.geojson')
gdf_rails = gpd.read_file(r'../data/warren/osm_warren/OSM_Railways.geojson')
gdf_boundary = gpd.read_file(glob.glob(r'../data/warren/boundary_buffer*.geojson')[0])

gdf_boundary.plot(ax=ax, color='grey')
gdf_roads.plot(ax=ax, color='white', linewidth=1)
gdf_rails.plot(ax=ax, color='black', linewidth=2)

ax.set_axis_off()
ax.set_title('Warren County, Open Street Maps Dataset\n(Roads & Railways)', y=0.95)
plt.tight_layout()
plt.show()

In [ ]:
###########################################
# Visualize Roads & Railways (Images) 
###########################################

roads_path = r'../data/warren/OSM_Roads.tif'

rails_path = r'../data/warren/OSM_Railways.tif'

fig, ax = plt.subplots(ncols=2, figsize=(12,6))

with rasterio.open(roads_path, 'r') as roads:
    show(roads, ax=ax[0])

with rasterio.open(rails_path, 'r') as rails:
    show(rails, ax=ax[1])

for axes in ax:
    axes.set_axis_off()
ax[0].set_title('Roads', style='italic')
ax[1].set_title('Railways', style='italic')
plt.suptitle('Open Street Maps Dataset - Warren County', y=0.95)
plt.tight_layout()
plt.show()

## Hardin County

# Patches

## Warren County

In [ ]:
from shapely.geometry import box

def create_patches(boundary_path, patch_size, overlap, image_resolution, output_path):
    """
    Function to create a grid of square patches of specified size covering an area. Patches are square and assume that output will be used to mask/clip images for downstream tasks.

    Parameters
    ----------
    boundary_path : string
        Path to area vector file (.shp or .geojson).
    patch_size : int
        Size of patches in pixels. Final patch is square (patch_size x patch_size).
    overlap : float
        Proportion of overlap of neighboring patches.
    image_resolution : int or float
        Spatial resolution of pixels.
    output_path : string
        Path for output GeoJSON of patches.

    Returns
    -------
    None
    """
    gdf = gpd.read_file(boundary_path)
    gdf['geometry'] = gdf['geometry'].buffer(0)
    minx, miny, maxx, maxy = gdf.total_bounds

    patch_size_spatial_units = patch_size * image_resolution
    overlap_patch_start = patch_size_spatial_units * (1-overlap)

    patches = []
    x = minx
    while x < maxx:
        y = miny
        while y < maxy:
            patch = box(x, y, x+patch_size_spatial_units, y+patch_size_spatial_units)
            if patch.intersects(gdf.unary_union):
                patches.append(patch)
            y += overlap_patch_start
        x += overlap_patch_start
    
    gdf_patches = gpd.GeoDataFrame(crs=gdf.crs, geometry=patches)
    # gdf_patches['id'] = [f"{patch_size}_{i}" for i in range(1, len(gdf_patches)+1)]
    # gdf_final = gdf_patches.sjoin(gdf, how='inner', predicate='intersects')
    gdf_final = gpd.sjoin(gdf_patches, gdf, how='inner', predicate='intersects')
    # gdf_final = gdf_patches.overlay(gdf, how='intersection')
    gdf_final.to_file(output_path, driver='GeoJSON')


In [ ]:
def get_patch_attributes_and_clean(patches_path, image_path, patch_size, output_path):
    """
    Function to create unique ID for patches, collect attributes from a given image, and save as new GeoJSON.

    Parameters
    ----------

    Returns
    -------
    None
    """
    gdf = gpd.read_file(patches_path)

    # collect attributes
    attributes = []
    with rasterio.open(image_path) as src:
        for idx, row in gdf.iterrows():
            
            ##### use rasterstats
            
            # get classes present in patch

            # calculate proportions of classes

            # create dictionary of class:proportion

            # append to list

            pass
    
    # append list of dictionaries to patches geodataframe
    
    # delete patches without any attributes (no overlap with image)

    # create unique IDs
    gdf['id'] = [f"{patch_size}_{i}" for i in range(1, len(gdf)+1)]

    # save as new GeoJSON

    pass

In [ ]:
# path to warren county dataset boundary (not buffered boundary)
boundary_path = r'../data/warren/boundary.geojson'

# define patch size (in pixels)
patch_size = 256

# define overlap size (in proportion)
overlap = 0.5

# spatial resolution of reference image
image_resolution = 5

# output path of patches GeoJSON
output_path = f"../data/warren/patches_{patch_size}.geojson"


##### create patches and save as GeoJSON
create_patches(boundary_path, patch_size, overlap, image_resolution, output_path)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

boundary = gpd.read_file(r'../data/warren/boundary.geojson')
patches = gpd.read_file(r'../data/warren/patches_256.geojson')

boundary.plot(ax=ax)
patches.plot(ax=ax, edgecolor='k', facecolor='none', linewidth=0.5)

ax.set_axis_off()
ax.set_title(f"Warren County Dataset Image Patches\n(256x256, 50% overlap, n={len(patches)})", y=0.95)
plt.show()

## Hardin County

# Verify Image Alignment

In [ ]:
# paths to images
image_paths = glob.glob(r'../data/warren/*.tif')
image_paths.sort()

for image in image_paths:
    with rasterio.open(image) as src:
        print('Name: ', os.path.basename(image))
        print('Spatial resolution: ', src.res)
        print('Width (pixels): ', src.meta['width'])
        print('Height (pixels): ', src.meta['height'])
        print(src.bounds)
        print('\n')


# Scratch...

In [ ]:
path = r'../data/warren/aerial_tiles/N158E178_2021.tif'

with rasterio.open(path) as src:
    data = src.read(4, masked=True)
    print(src.nodata)
    print(src.meta['dtype'])
    print(np.min(data))
    print(np.max(data))
    show(data)


clip_image_to_boundary(path, boundary_path, output_tif_path=r'../data/warren/test.tif')

with rasterio.open(r'../data/warren/test.tif') as src:
    data = src.read(4, masked=True)
    print(src.nodata)
    print(src.meta['dtype'])
    print(np.min(src.read(1)))
    print(np.max(src.read(1)))
    show(data)





In [ ]:
path = r'/Users/matthew/GitHub/cs612/data/warren/dem.tif'
# path = r'/Users/matthew/GitHub/cs612/data/warren/dem_tiles/N176E177.tif'
with rasterio.open(path) as src:
    print(src.nodata)
    print(src.meta['dtype'])
    data = src.read(1, masked=True)
    print(np.min(data))
    print(np.max(data))
    show(data)

In [ ]:
output_aerial_path = r'../data/warren/aerial.tif'

fig, ax = plt.subplots()
with rasterio.open(output_aerial_path, tiled=True, blockxsize=1024, blockysize=1024) as src:
    show(src, ax=ax)

In [ ]:
# import glob
# import os
# aerial_tile_dir = r'../data/warren/aerial_tiles'

clipped_edge_tile_paths = glob.glob(f"{aerial_tile_dir}/*clip.tif")
clipped_edge_tile_paths
for path in clipped_edge_tile_paths:
    os.remove(path)

In [ ]:
##################################################
# merge aerial imagery tiles into single geotiff
##################################################

# directory containing aerial imagery tiles
aerial_dir = r'../data/warren/aerial_tiles'

# output path for single mosaiced aerial image
output_aerial_path = r'../data/warren/aerial_mosaic.tif'

# custom function to mosaic aerial image tiles and save as new single image
mosaic_image_tiles(aerial_dir, output_aerial_path)

In [ ]:
#########################################
# clip dem to warren county area extent
#########################################

# path to input mosaiced dem from above
input_im_path = r'../data/warren/aerial_mosaic.tif'

# path to geodatabase
gdb_path = r'../data/geology.gdb'

# layer name of boundary feature class in geodatabase used to clip dem 
boundary_layer = r'warren_geo_boundary'

# output path of new clipped dem
output_im_path = r'../data/warren/aerial.tif'

# call custom function to clip mosaiced dem to outline of labeled dataset boundary
clip_image_to_boundary(input_im_path, gdb_path, boundary_layer, output_im_path)

In [ ]:
##########################
# visualize clipped dem
##########################

fig, ax = plt.subplots(figsize=(10,10))

with rasterio.open(output_dem_path) as src:
    data = src.read(1, masked=True)
    ax.imshow(data, cmap='viridis', interpolation='nearest')
    ax.set_title('Warren County DEM')
    ax.set_xlabel('X (pixels)')
    ax.set_ylabel('Y (pixels)')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_position(('outward', 10))  # Offset bottom axis
    ax.spines['left'].set_position(('outward', 10))    # Offset left axis
    # fig.colorbar(dem, ax=ax, orientation='vertical')
    plt.tight_layout()
    plt.show()